In [72]:
%load_ext autoreload
%autoreload 2

# format all data.json files needed for running with LAUG files 
# 1. replace all text with text from laug text (TP & SD)
# 2. replace all text with mutated entities and change labels with mutated entities

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# create dataset with named entities replaced 

In [73]:
from parlai.tasks.multiwoz_checkdst.mutator import EntityMutator

In [74]:
named_entity_slots_for_trippy ={
    "attraction-name",
    "restaurant-name",
    "hotel-name",
    "taxi-departure",
    "taxi-destination",
    "train-departure",
    "train-destination",
}

In [75]:
import numpy as np  
from argparse import ArgumentParser

In [76]:
# enforce same mutation as ParlAI agent 
SEED = 42
rng = np.random.RandomState(SEED)
mutator = EntityMutator(opt=None, rng=rng)

In [77]:
sample =    {'dialog_act': {'Booking-Request': [['Stay', '?'], ['Day', '?']]},
    'metadata': {'attraction': {'book': {'booked': []},
      'semi': {'area': '', 'name': '', 'type': ''}},
     'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
     'hotel': {'book': {'booked': [],
       'day': 'monday',
       'people': '',
       'stay': ''},
      'semi': {'area': 'centre',
       'internet': 'dontcare',
       'name': 'alexander bed and breakfast',
       'parking': 'not mentioned',
       'pricerange': 'not mentioned',
       'stars': 'not mentioned',
       'type': 'guesthouse'}},
     'police': {'book': {'booked': []}, 'semi': {}},
     'restaurant': {'book': {'booked': [],
       'day': '',
       'people': '',
       'time': ''},
      'semi': {'area': '', 'food': '', 'name': '', 'pricerange': ''}},
     'taxi': {'book': {'booked': []},
      'semi': {'arriveBy': '',
       'departure': '',
       'destination': '',
       'leaveAt': ''}},
     'train': {'book': {'booked': [], 'people': ''},
      'semi': {'arriveBy': '',
       'day': '',
       'departure': '',
       'destination': '',
       'leaveAt': ''}}},
    'span_info': [],
    'text': 'what day would you like to book on and for how long ?',
    'turn_id': 5}

In [78]:
from utils import replace_dict_values_str

In [79]:
# replace_dict_values_str(sample, "alexander bed and breakfast", "some sample word")
replace_dict_values_str(sample, "alexander bed and breakfast", "whatever")
sample

{'dialog_act': {'Booking-Request': [['Stay', '?'], ['Day', '?']]},
 'metadata': {'attraction': {'book': {'booked': []},
   'semi': {'area': '', 'name': '', 'type': ''}},
  'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
  'hotel': {'book': {'booked': [], 'day': 'monday', 'people': '', 'stay': ''},
   'semi': {'area': 'centre',
    'internet': 'dontcare',
    'name': 'whatever',
    'parking': 'not mentioned',
    'pricerange': 'not mentioned',
    'stars': 'not mentioned',
    'type': 'guesthouse'}},
  'police': {'book': {'booked': []}, 'semi': {}},
  'restaurant': {'book': {'booked': [], 'day': '', 'people': '', 'time': ''},
   'semi': {'area': '', 'food': '', 'name': '', 'pricerange': ''}},
  'taxi': {'book': {'booked': []},
   'semi': {'arriveBy': '',
    'departure': '',
    'destination': '',
    'leaveAt': ''}},
  'train': {'book': {'booked': [], 'people': ''},
   'semi': {'arriveBy': '',
    'day': '',
    'departure': '',
    'destination': '',
    'leaveAt': 

In [80]:
import json

In [81]:
orig_fn = "multiwoz_dst/MULTIWOZ2.3/data.json"
with open(orig_fn, "r") as f: 
    orig = json.load(f)

In [82]:
from tqdm import tqdm 

In [83]:
with open("multiwoz_dst/MULTIWOZ2.1/testListFile.json", "r") as f: 
    test_convs = f.readlines()
    test_convs = [t.replace("\n", "") for t in test_convs]

In [84]:
len(test_convs)

1000

In [85]:
list(orig.keys())[0]

'SNG01856.json'

In [86]:
test_convs[0]

'MUL0484.json'

In [87]:
swapped_ct = 0 
change = False 
ned_conversion_map = {} 
nei_orig = orig.copy()  
slot_vals = set()
change_list = [] 
n_text_examples = 0 
print_ = False 
for dial_id, dial in tqdm(nei_orig.items()): 

    def replace_with_scrambled_entity(domain, slot_type, slot_value, print_=False): 
        slot_value = slot_value.lower() 
        change = False 
        if f"{domain}-{slot_type}" in named_entity_slots_for_trippy: 
            change = True

            if "".join(sorted(slot_value)) not in ned_conversion_map: 
                new_v  = mutator.scramble(slot_value)
                ned_conversion_map["".join(sorted(slot_value))] = new_v
            else: 
                new_v = ned_conversion_map["".join(sorted(slot_value))]

            # go through all turns and replace if found 
            for idx2 in range(len(dial['log'])): 
                # update only if the golden slot value is actually found in the text
                if slot_value.lower() in dial['log'][idx2]['text'].lower(): 
                    # update text 
                    dial['log'][idx2]['text']= dial['log'][idx2]['text'].lower().replace(slot_value, new_v)
                    # update metadata 
                    replace_dict_values_str(dial['log'][idx2]['metadata'], slot_value, new_v)
                    change = True 

                    if print_: 
                        print(dial['log'][idx2]['text'])
                        print(f"{domain} {slot_type} {slot_value}")
                        print(f"{domain} {slot_type} {new_v}")
                
        return change 
    
    for idx in reversed(range(0, len(dial['log']), 2)): 
        user_utt = dial['log'][idx]['text']
        sys_utt = dial['log'][idx+1]['text']
        metadata = dial['log'][idx+1]['metadata']

        change = False 
        for domain, slots in metadata.items(): 
            for k, v in slots['book'].items(): 
                if k == "booked": 
                    if v == []: 
                        continue 
                    else: 
                        for booked_places in slots['book']['booked']: 
                            for slot_type, slot_value in booked_places.items():
                                slot_vals.add(slot_value)
                                change = change or replace_with_scrambled_entity(domain, slot_type, slot_value, print_=print_)
                                
                        
                elif v and v!= "not mentioned":  
                    # print(f"{domain} {k} {v}")
                    change = change or replace_with_scrambled_entity(domain, k, v, print_=print_)

            for k,v in slots['semi'].items(): 
                if v and v!="not mentioned": 
                    change = change or replace_with_scrambled_entity(domain, k, v, print_=print_)
        n_text_examples += dial_id in test_convs 
        if change and dial_id in test_convs:
            dial_full_id = f"{dial_id.lower().replace('.json', '')}-{idx//2}"
            change_list.append(dial_full_id) 

        # break 
    

100%|██████████| 10438/10438 [00:04<00:00, 2194.66it/s]


In [88]:
n_text_examples

7372

In [89]:
len(change_list)

6055

In [90]:
assert n_text_examples == 7372 

In [91]:
sorted(change_list)[0]

'mul0003-3'

In [92]:
data_dir = os.environ.get("DATADIR", "")

with open(os.path.join(data_dir, "checkdst/NED/changed_dialids_test_fewshot_False.txt"), "r") as f: 
    changed_parlai_ids = f.readlines()
    changed_parlai_ids = [c.replace("\n", "") for c in changed_parlai_ids]

In [93]:
sorted(change_list)[0], sorted(changed_parlai_ids)[0]

('mul0003-3', 'mul0003-5')

In [94]:
len(changed_parlai_ids), len(change_list), len(set(change_list).intersection(changed_parlai_ids))

(5804, 6055, 5804)

In [95]:
len(ned_conversion_map)

1076

In [96]:
ne_samples = [
    "birmingham new street", 
    "golden house", 
    "express by holiday inn cambridge",
    "the cambridge belfry"
]

for ne_sample in ne_samples: 
    print("".join(sorted(ne_sample.lower())) in ned_conversion_map)

True
True
True
True


In [97]:
#need to update the dialogue_acts.json file as well?? for inference, no. 
with open("multiwoz_dst/MULTIWOZ2.1/dialogue_acts.json", "r") as f: 
    dacts = json.load(f)

In [98]:
ACTS_DICT = {'taxi-depart': 'taxi-departure',
             'taxi-dest': 'taxi-destination',
             'taxi-leave': 'taxi-leaveAt',
             'taxi-arrive': 'taxi-arriveBy',
             'train-depart': 'train-departure',
             'train-dest': 'train-destination',
             'train-leave': 'train-leaveAt',
             'train-arrive': 'train-arriveBy',
             'train-people': 'train-book_people',
             'restaurant-price': 'restaurant-pricerange',
             'restaurant-people': 'restaurant-book_people',
             'restaurant-day': 'restaurant-book_day',
             'restaurant-time': 'restaurant-book_time',
             'hotel-price': 'hotel-pricerange',
             'hotel-people': 'hotel-book_people',
             'hotel-day': 'hotel-book_day',
             'hotel-stay': 'hotel-book_stay',
             'booking-people': 'booking-book_people',
             'booking-day': 'booking-book_day',
             'booking-stay': 'booking-book_stay',
             'booking-time': 'booking-book_time',
}


In [99]:
def load_acts(input_file):
    with open(input_file) as f:
        acts = json.load(f)
    s_dict = {}
    for d in acts:
        for t in acts[d]:
            # Only process, if turn has annotation
            if isinstance(acts[d][t], dict):
                for a in acts[d][t]:
                    aa = a.lower().split('-')
                    if aa[1] == 'inform' or aa[1] == 'recommend' or aa[1] == 'select' or aa[1] == 'book':
                        for i in acts[d][t][a]:
                            s = i[0].lower()
                            v = i[1].lower().strip()
                            if s == 'none' or v == '?' or v == 'none':
                                continue
                            slot = aa[0] + '-' + s
                            if slot in ACTS_DICT:
                                slot = ACTS_DICT[slot]
                            key = d + '.json', t, slot
                            # In case of multiple mentioned values...
                            # ... Option 1: Keep first informed value
                            if key not in s_dict:
                                s_dict[key] = list([v])
                            # ... Option 2: Keep last informed value
                            #s_dict[key] = list([v])
    return s_dict

In [100]:
s_dict = load_acts("multiwoz_dst/MULTIWOZ2.3/dialogue_acts.json")

In [101]:
from loguru import logger

In [115]:
from collections import Counter 
slot_counter = Counter()
slotname_counter = Counter()
slotvalue_counter = Counter()
count = 0 
for dial_id, turn_acts in tqdm(dacts.items()): 
    # print(dial_id + ".json")
    # break

    if dial_id + ".json" not in test_convs: 
        continue
    else: 
        count += 1

    for turn, acts in turn_acts.items(): 

        if isinstance(acts, dict): 
            for slot, slot_vals in acts.items(): 
                for sv in slot_vals: 
                    # print(slot, sv)
                    slot_counter[f"{slot}-{sv[0]}-{sv[1]}"] += 1 
                    slotname_counter[f"{slot}-{sv[0]}"] += 1 
                    

                    if f"{slot}-{sv[0]}" == "Restaurant-Inform-Name" and sv[1] != "None": 
                        slotvalue_counter[sv[0]] += 1 
                        if "".join(sorted(sv[1].lower())) not in ned_conversion_map: 
                            logger.info((sv[1], "".join(sorted(sv[1].lower()))))
                        # assert "".join(sorted(sv[1].lower())) in nei_conversion_map, (sv[1], "".join(sorted(sv[1].lower())))
        else: 
            if acts == "No Annotation": 
                continue 
            print(f"Not a dict: {acts}")

    # print(acts)
    # break
count

  0%|          | 0/10438 [00:00<?, ?it/s]2022-03-08 01:28:39.116 | INFO     | __main__:<module>:28 - ("prezzo's", "'eoprszz")
2022-03-08 01:28:39.117 | INFO     | __main__:<module>:28 - (' anatolia restauran', '  aaaaaeilnnorrsttu')
2022-03-08 01:28:39.121 | INFO     | __main__:<module>:28 - (' the river bar steakhouse and grill', '      aaabdeeeeghhiikllnorrrrssttuv')
2022-03-08 01:28:39.123 | INFO     | __main__:<module>:28 - (' the golden curry', '   cdeeghlnorrtuy')
2022-03-08 01:28:39.124 | INFO     | __main__:<module>:28 - (' saffron brasserie', '  aabeeffinorrrsss')
2022-03-08 01:28:39.130 | INFO     | __main__:<module>:28 - (' Riverside Brasserie', '  abdeeeeiiirrrrsssv')
2022-03-08 01:28:39.139 | INFO     | __main__:<module>:28 - (' Rice House', '  ceehiorsu')
2022-03-08 01:28:39.140 | INFO     | __main__:<module>:28 - (' Golden House', '  deeghlnoosu')
2022-03-08 01:28:39.142 | INFO     | __main__:<module>:28 - ('curry prince ', '  cceinprrruy')
2022-03-08 01:28:39.144 | INFO

1000

In [116]:
slot_counter.most_common(10)
slotvalue_counter.most_common(10)

[('Name', 359)]

In [104]:
orig_fn

'multiwoz_dst/MULTIWOZ2.3/data.json'

In [105]:
nei_orig_fn = "/data/home/justincho/trippy-public-master/data/laug_dst/NEI/data.json"
with open(nei_orig_fn, "w") as f: 
    json.dump(nei_orig, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '/data/home/justincho/trippy-public-master/data/laug_dst/NEI/data.json'

In [107]:
orig_fn_test = "multiwoz_dst/MULTIWOZ2.3/test_dials.json"


In [108]:
with open(orig_fn_test, "r") as f: 
    orig_test_data = json.load(f)

In [109]:
list(nei_orig.keys())[0], list(orig_test_data.keys())[0]

('SNG01856.json', 'MUL0003.json')

In [110]:
nei_test_data = {} 
for k in orig_test_data.keys(): 
    nei_test_data[k] = nei_orig[k]

In [111]:
len(nei_test_data)

1000

In [112]:
str_data = json.dumps(nei_test_data)

In [114]:
"Golden House".lower() in str_data.lower()

True

In [ ]:
nei_test_fn = "checkdst/NED/test_dials.json"
with open(nei_test_fn, "w") as f: 
    json.dump(nei_test_data, f, indent=4)

In [ ]:
with open(nei_test_fn, "r") as f: 
    data = f.read()

In [ ]:
data[0]

'{'

In [ ]:
nei_test_data['MUL0003.json']['log']

[{'text': "i 'm looking for a place to stay . it needs to be a guesthouse and include free wifi .",
  'metadata': {},
  'dialog_act': {'Hotel-Inform': [['Type', 'guesthouse'],
    ['Internet', 'yes']]},
  'span_info': [['Hotel-Inform', 'Type', 'guesthouse', 14, 14]],
  'turn_id': 0},
 {'text': 'there are 23 hotels that meet your needs . would you like to narrow your search by area and/or price range ?',
  'metadata': {'taxi': {'book': {'booked': []},
    'semi': {'leaveAt': '',
     'destination': '',
     'departure': '',
     'arriveBy': ''}},
   'police': {'book': {'booked': []}, 'semi': {}},
   'restaurant': {'book': {'booked': [], 'people': '', 'day': '', 'time': ''},
    'semi': {'food': '', 'pricerange': '', 'name': '', 'area': ''}},
   'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
   'hotel': {'book': {'booked': [], 'people': '', 'day': '', 'stay': ''},
    'semi': {'name': 'not mentioned',
     'area': 'not mentioned',
     'parking': 'not mentioned',
     

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv("/data/home/justincho/trippy-public-master/results/of_interest/multiwoz23_lr1e-4_2022-01-05_09:41:24_fewshot_False_46/pred_res.test.finalNEI.csv")

In [ ]:
NAMED_ENTITY_SLOTS_FOR_TRIPPY ={
    "attraction-name",
    "restaurant-name",
    "hotel-name",
    "taxi-departure",
    "taxi-destination",
    "train-departure",
    "train-destination",
}


In [ ]:
from metric_bert_dst import load_dataset_config, tokenize, check_slot_inform

In [ ]:
dataset_config = "dataset_config/multiwoz21.json"
class_types, slots, label_maps = load_dataset_config(dataset_config)

In [ ]:
from collections import Counter

In [ ]:
jgas = [] 
ne_jga =[]
ct =0 
false_pred_ct = 0 
false_pred_dict= {} 
counter = Counter() 
print_ct =  0 
for idx, row in df.iterrows(): 
    found_ne_slot = False
    for ne_slot in NAMED_ENTITY_SLOTS_FOR_TRIPPY: 
        if row[f"{ne_slot}_gold"] != "none": 
            found_ne_slot = True 
            break

    if found_ne_slot: 
        jga = int(all([row[f"{slot}_gold"] == row[f"{slot}_pred"] for slot in slots]))
        jgas.append(jga)

    for ne_slot in NAMED_ENTITY_SLOTS_FOR_TRIPPY: 
        # if row[f"{ne_slot}_gold"] != "none":
        #     continue
        # if row[f"{ne_slot}_gold"] == "none" and row[f"{ne_slot}_pred"] != "none": 
        #     # print(f"{}")
        #     continue
        if row[f"{ne_slot}_gold"] != "none" and row[f"{ne_slot}_pred"] == row[f"{ne_slot}_gold"] : 
            # print(row[f"{ne_slot}_gold"])
            # print(row[f"{ne_slot}_pred"])
            counter[row[f"{ne_slot}_gold"]] +=1 
            ct += 1 
        if row[f"{ne_slot}_gold"] != "none" and row[f"{ne_slot}_pred"] != "none" and row[f"{ne_slot}_pred"] != row[f"{ne_slot}_gold"] : 
            if print_ct < 100: 
                print(f"{row['id']} \n\tpred: " + row[f"{ne_slot}_pred"], "\n\tgold: " + row[f"{ne_slot}_gold"])
                print_ct += 1 
            
        if row[f"{ne_slot}_gold"] == "none" and row[f"{ne_slot}_pred"] != "none": 
            false_pred_ct += 1 
            # print()


np.mean(jgas), len(jgas), ct, false_pred_ct  
    # are there many casese where there were no named entities yet one was predicted for in the augmented set? 

mul0011.json-2 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-3 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-4 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-5 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-6 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-7 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-8 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-9 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-10 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-11 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-11 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-12 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-12 
	pred: cafe 
	gold: lcoafns wec
mul0014.json-4 
	pred: lrhtaa ein 
	gold: arianl
mul0014.json-5 
	pred: lrhtaa ein 
	gold: arianl
mul0021.json-3 
	pred: the oehkc niaa t nzpicdrwza b [ SEP ] absolutely ! enlf etoilhhestd le is a [ UNK ] star hotel and their phone number is 01 223 355017 . is there anything else i can help you with ? [ SEP ] i ' d just need their star rating and phone number , thank y

(0.04690338563170933, 6055, 2275, 71)

In [ ]:
from pprint import pprint

In [ ]:
def show_concise_metadata(metadata_dict): 
    concise = {} 
    for domain, slots in metadata_dict.items(): 
        concise_domain = {} 
        for slot_type, slotkv in slots.items(): 
            concise_slotkv = {} 
            for k,v in slotkv.items(): 
                if v: 
                    concise_slotkv[k] = v 
            if concise_slotkv: 
                concise_domain[slot_type] = concise_slotkv 
        if concise_domain: 
            concise[domain] = concise_domain 
    if concise: 
        return concise 

In [ ]:
for t in nei_test_data["MUL0594.json"]["log"]: 
    print(t["text"])
    # pprint(t["metadata"])
    if t["metadata"]: 
        pprint(show_concise_metadata(t["metadata"]))

hello , i need to book a ticket going to reamcgbdi arriving by 16:30 , can you help me with that ?
there are a lot of trains available . where are you departing from ?
{'train': {'semi': {'arriveBy': '16:30',
                    'day': 'not mentioned',
                    'departure': 'not mentioned',
                    'destination': 'reamcgbdi',
                    'leaveAt': 'not mentioned'}}}
i 'm departing from norurobbex on friday .
tr2145 looks like it will get you there with plenty of time to spare .
{'train': {'semi': {'arriveBy': '16:30',
                    'day': 'friday',
                    'departure': 'norurobbex',
                    'destination': 'reamcgbdi',
                    'leaveAt': 'not mentioned'}}}
thanks , could you let me know the departure and arrival time ?
of course . it was actually the tr2144 , my apologies about that .. it leaves at 14:32 and gets you there at about 15:32 . it costs 17.90 pounds .
{'train': {'semi': {'arriveBy': '16:30',
          